In [3]:
! python -m pip install google-genai lxml numpy

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.3 MB 9.3 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/12.3 MB 8.3 MB/s eta 0:00:02
   --------------- ------------------------ 4.7/12.3 MB 8.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.8/12.3 MB 8.8 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.3 MB 9.2 MB/s eta 0:00:01
   ------------------------------------- -- 11.5/12.3 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 9.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from lxml import etree
from google.genai import client, types
from google import genai
import numpy as np

api_key = ''
client = genai.Client(api_key=api_key)

In [23]:
file_path = "xsl_files/courses.xsl"
file_type = "xsl"

In [24]:
tree = etree.parse(file_path)
root = tree.getroot()

In [25]:
def chunks_xml(element, path=""):
    current_path = f"{path}/{element.tag}"
    text = f"Element Path: {current_path}\nTag: {element.tag}\nAttributes: {element.attrib}\n"
    if element.text and element.text.strip():
        text += f"Text: {element.text.strip()}\n"
    
    for child in element:
        if child.text and child.text.strip():
            text += f"{child.tag}: {child.text.strip()}\n"
        if child.attrib:
            text += f"{child.tag} attributes: {child.attrib}\n"

    chunks = [text.strip()]
    
    for child in element:
        chunks.extend(chunks_xml(child, current_path))
    return chunks

In [29]:
def chunks_xsl(element, path=""):
    full_tag = str(element.tag)

    if "}" in full_tag:
        ns, tag = full_tag[1:].split("}", 1)
        if ns.endswith("1999/XSL/Transform"):
            full_tag = f"xsl:{tag}"
        elif ns.endswith("1999/XSL/Format"):
            full_tag = f"fo:{tag}"
        else:
            full_tag

    current_path = f"{path}/{full_tag}"
    text = f"Element Path: {current_path}\nTag: {full_tag}\n"
    
    if element.attrib:
        text += f"Attributes: {element.attrib}\n"
    if element.text and element.text.strip():
        text += f"Text: {element.text.strip()}\n"

    for child in element:
        if child.text and child.text.strip():
            text += f"{child.tag}: {child.text.strip()}\n"
        if child.attrib:
            text += f"{child.tag} attributes: {child.attrib}\n"
    
    chunks = [text.strip()]

    for child in element:
        chunks.extend(chunks_xsl(child, current_path))
    return chunks

In [30]:
if file_type == "xml":
    chunks = chunks_xml(root)
elif file_type == "xsl":
    chunks = chunks_xsl(root)
    
print("Filetype: ", file_type)
print("Amount of chunks: ", len(chunks))

Filetype:  xsl
Amount of chunks:  24
